# Extended Kalman filter for 3 DOF linear model
An Extended Kalman filter with a 3 DOF linear model as the predictor will be developed.
The filter is run on simulated data as well as real model test data.

In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from numpy.linalg import inv
import sympy as sp

import src.visualization.book_format as book_format
book_format.set_style()
from src.substitute_dynamic_symbols import lambdify
from sympy import Matrix
from sympy.physics.mechanics import (dynamicsymbols, ReferenceFrame,
                                      Particle, Point)
from IPython.display import display, Math, Latex
from src.substitute_dynamic_symbols import run, lambdify
from sympy.physics.vector.printing import vpprint, vlatex
from src.data import mdl
from src.extended_kalman_filter import extended_kalman_filter, rts_smoother
import src.models.vmm_nonlinear_EOM  as vmm
from docs.book.example_1 import ship_parameters, df_parameters
from src.symbols import *
from src import prime_system
p = df_parameters["symbol"]
from src.visualization.plot import track_plot, plot

import matplotlib.pyplot as plt
import os
if os.name == 'nt':
    plt.style.use('../docs/book/book.mplstyle')  # Windows

Duplicate key in file WindowsPath('C:/Users/maa/.matplotlib/stylelib/paper.mplstyle'), line 462 ('figure.figsize   : 5, 3   ## figure size in inches')
Duplicate key in file WindowsPath('C:/Users/maa/.matplotlib/stylelib/paper.mplstyle'), line 463 ('figure.dpi       : 100        ## figure dots per inch')
C:\Dev\wPCC\venv\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


## 3DOF model

In [2]:
X_eq = vmm.X_eq
Y_eq = vmm.Y_eq
N_eq = vmm.N_eq

A, b = sp.linear_eq_to_matrix([X_eq, Y_eq, N_eq], [u1d, v1d, r1d])

acceleration = sp.matrices.MutableDenseMatrix([u1d,v1d,r1d])
eq_simulator = sp.Eq(sp.UnevaluatedExpr(A)*sp.UnevaluatedExpr(acceleration),sp.UnevaluatedExpr(b))
eq_simulator

Eq(Matrix([
[-X_{\dot{u}} + m,                    0,                    0],
[               0,     -Y_{\dot{v}} + m, -Y_{\dot{r}} + m*x_G],
[               0, -N_{\dot{v}} + m*x_G,    I_z - N_{\dot{r}}]])*Matrix([
[\dot{u}],
[\dot{v}],
[\dot{r}]]), Matrix([
[-m*(-r**2*x_G - r*v) + X_D(u, v, r, delta)],
[              -m*r*u + Y_D(u, v, r, delta)],
[          -m*r*u*x_G + N_D(u, v, r, delta)]]))

In [3]:
A_inv = A.inv()
S = sp.symbols('S')
eq_S=sp.Eq(S,-sp.fraction(A_inv[1,1])[1])

A_inv_S = A_inv.subs(eq_S.rhs,S)
eq_acceleration_matrix_clean = sp.Eq(sp.UnevaluatedExpr(acceleration),sp.UnevaluatedExpr(A_inv_S)*sp.UnevaluatedExpr(b))
Math(vlatex(eq_acceleration_matrix_clean))

<IPython.core.display.Math object>

In [4]:
u1d_function = sp.Function(r'\dot{u}')(u,v,r,delta)
v1d_function = sp.Function(r'\dot{v}')(u,v,r,delta)
r_function = sp.Function(r'\dot{r}')(u,v,r,delta)

subs_prime = [
    
    (m,m/prime_system.df_prime.mass.denominator),
    (I_z,I_z/prime_system.df_prime.inertia_moment.denominator),
    (x_G,x_G/prime_system.df_prime.length.denominator),
    
    (u, u/sp.sqrt(u**2+v**2)),
    (v, v/sp.sqrt(u**2+v**2)),
    (r, r/(sp.sqrt(u**2+v**2)/L)),
   
]

subs = [
(X_D, vmm.X_qs_eq.rhs),
(Y_D, vmm.Y_qs_eq.rhs),
(N_D, vmm.N_qs_eq.rhs),
]

subs = subs + subs_prime

A_SI = A.subs(subs)
b_SI = b.subs(subs)

x_dot = sympy.matrices.dense.matrix_multiply_elementwise(A_SI.inv()*b_SI,
                                                 sp.Matrix([(u**2+v**2)/L,(u**2+v**2)/L,(u**2+v**2)/(L**2)]))


In [7]:
x_dot

Matrix([
[                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          1.0*L**2*rho*(u**2 + v**2)*(L**2*X_{rr}*r**2/(u**2 + v**2) + L*X_{r}*r/sqrt(u**2 + v**2) + L*X_{vr}*r*v/(u**2 + v**2) + X_{delta}*delta + X_{u}*u/sqrt(u**2 + v**2/(u**2 + v**2)) + 

In [14]:
x_dot[1].args[2]

(-4.0*I_z*L**3*rho + 2.0*L**8*N_{\dot{r}}*rho**2)*(L*Y_{r}*r/sqrt(u**2 + v**2) + L*Y_{ur}*r*u/(sqrt(u**2 + v**2)*sqrt(u**2 + v**2/(u**2 + v**2))) + Y_{delta}*delta + Y_{u}*u/sqrt(u**2 + v**2/(u**2 + v**2)) + Y_{v}*v/sqrt(u**2 + v**2) - 2.0*m*r*u/(L**2*rho*sqrt(u**2 + v**2)*sqrt(u**2 + v**2/(u**2 + v**2))))/(4.0*I_z*L**3*Y_{\dot{v}}*rho - 8.0*I_z*m - 2.0*L**8*N_{\dot{r}}*Y_{\dot{v}}*rho**2 + 2.0*L**8*N_{\dot{v}}*Y_{\dot{r}}*rho**2 + 4.0*L**5*N_{\dot{r}}*m*rho - 4.0*L**4*N_{\dot{v}}*m*rho*x_G - 4.0*L**4*Y_{\dot{r}}*m*rho*x_G + 8.0*m**2*x_G**2) + (-2.0*L**8*Y_{\dot{r}}*rho**2 + 4.0*L**4*m*rho*x_G)*(L*N_{r}*r/sqrt(u**2 + v**2) + L*N_{ur}*r*u/(sqrt(u**2 + v**2)*sqrt(u**2 + v**2/(u**2 + v**2))) + N_{delta}*delta + N_{u}*u/sqrt(u**2 + v**2/(u**2 + v**2)) + N_{v}*v/sqrt(u**2 + v**2) - 2.0*m*r*u*x_G/(L**3*rho*sqrt(u**2 + v**2)*sqrt(u**2 + v**2/(u**2 + v**2))))/(4.0*I_z*L**3*Y_{\dot{v}}*rho - 8.0*I_z*m - 2.0*L**8*N_{\dot{r}}*Y_{\dot{v}}*rho**2 + 2.0*L**8*N_{\dot{v}}*Y_{\dot{r}}*rho**2 + 4.0*L**5

In [75]:
x_ = sp.Matrix([u*sp.cos(psi)-v*sp.sin(psi),
                                u*sp.sin(psi)+v*sp.cos(psi),
                                r])

f_ = sp.Matrix.vstack(x_, x_dot)

subs = {value: key for key, value in p.items()}
subs[psi] = sp.symbols('psi')
lambda_f = lambdify(f_.subs(subs))

In [76]:
%%time
expr=f_.subs(subs)
lambda_f = lambdify(expr)

Wall time: 5.1 s


In [84]:
subs = {value: key for key, value in p.items()}
keys = list(set(subs.keys()) & f_.free_symbols)
subs = {key : subs[key] for key in keys}
expr=f_.subs(subs)

In [85]:
subs

{X_{vr}: 'Xvr',
 X_{u}: 'Xu',
 Y_{\dot{v}}: 'Yvdot',
 Y_{\dot{r}}: 'Yrdot',
 Y_{delta}: 'Ydelta',
 Y_{r}: 'Yr',
 X_{r}: 'Xr',
 Y_{ur}: 'Yur',
 X_{rr}: 'Xrr',
 X_{delta}: 'Xdelta',
 Y_{u}: 'Yu',
 N_{v}: 'Nv',
 N_{\dot{v}}: 'Nvdot',
 N_{u}: 'Nu',
 N_{r}: 'Nr',
 Y_{v}: 'Yv',
 N_{ur}: 'Nur',
 X_{\dot{u}}: 'Xudot',
 X_{v}: 'Xv',
 N_{delta}: 'Ndelta',
 N_{\dot{r}}: 'Nrdot'}

In [83]:
%%time
subs[psi] = sp.symbols('psi')
expr=f_.subs(subs)
sp.lambdify(list(expr.free_symbols), expr)

Wall time: 978 ms


<function _lambdifygenerated(Xv, Xudot, I_z, delta, Xvr, r, Xrr, x_G, u, v, Nrdot, Xu, Yr, psi, Yvdot, Nr, L, Nu, Yrdot, Xdelta, Xr, Nv, Yur, Nur, Yu, m, Yv, rho, Ndelta, Nvdot, Ydelta)>

In [74]:
sp.lambdify(list(f_.free_symbols), f_)

<function _lambdifygenerated(Dummy_572, I_z, delta, _Dummy_563, Dummy_573, r, Dummy_569, x_G, Dummy_570, Dummy_568, u, v, Dummy_567, Dummy_566, Dummy_575, L, Dummy_576, Dummy_565, Dummy_578, Dummy_583, Dummy_579, Dummy_581, m, Dummy_582, rho, Dummy_564, Dummy_580, Dummy_577, Dummy_571, Dummy_574, Dummy_584)>

In [6]:
lambda_f

<function _lambdifygenerated(I_z, L, Ndelta, Nr, Nrdot, Nu, Nur, Nv, Nvdot, Xdelta, Xr, Xrr, Xu, Xudot, Xv, Xvr, Ydelta, Yr, Yrdot, Yu, Yur, Yv, Yvdot, delta, m, psi, r, rho, u, v, x_G)>

In [ ]:
def lambda_f_constructor(parameters, ship_parameters):
    
    def f(x, u):
        delta = u
        
        psi=x[2]
        u=x[3]
        v=x[4]
        r=x[5]
        
        x_dot = run(lambda_f, **parameters, **ship_parameters, psi=psi, u=u, v=v, r=r, delta=delta).reshape(x.shape)
        return x_dot
    
    return f

## Simulation

In [ ]:
def time_step(x_,u_):
    psi=x_[2]
    u=x_[3]
    v=x_[4]
    r=x_[5]
    delta = u_
    x_dot = run(lambda_f, **parameters, **ship_parameters, psi=psi, u=u, v=v, r=r, delta=delta).flatten()
    return x_dot

def simulate(x0,E, ws, t, us):
    
    simdata = np.zeros((6,len(t)))
    x_=x0
    
    Ed = h_ * E
            
    for i,(u_,w_) in enumerate(zip(us,ws)):
        
        w_ = w_.reshape(len(w_),1)
        x_dot = lambda_f_(x_,u_) + Ed @ w_
        
        x_=x_ + h_*x_dot
        
        simdata[:,i] = x_.flatten()
                
    df = pd.DataFrame(simdata.T, columns=["x0","y0","psi","u","v","r"], index=t)
    df.index.name = 'time'
    df['delta'] = us
    
    return df

In [ ]:
parameters=df_parameters['prime'].copy()

lambda_f_ = lambda_f_constructor(parameters=parameters, 
                                               ship_parameters=ship_parameters)

N_ = 4000

t_ = np.linspace(0,50,N_)
h_ = float(t_[1]-t_[0])

us = np.deg2rad(30*np.concatenate((-1*np.ones(int(N_/4)),
                                1*np.ones(int(N_/4)),
                                -1*np.ones(int(N_/4)),
                                1*np.ones(int(N_/4)))))

x0_ = np.array([[0,0,0,3,0,0]]).T
no_states = len(x0_)

np.random.seed(42)
E = np.array([
    [0,0,0],
    [0,0,0],
    [0,0,0],
    [1,0,0],
    [0,1,0],
    [0,0,1],
    ],
)
process_noise_u = 0.01
process_noise_v = 0.01
process_noise_r = np.deg2rad(0.01)

ws = np.zeros((N_,3))
ws[:,0] = np.random.normal(loc=process_noise_u, size=N_)
ws[:,1] = np.random.normal(loc=process_noise_v, size=N_)
ws[:,2] = np.random.normal(loc=process_noise_r, size=N_)

df = simulate(x0=x0_, E=E, ws=ws, t=t_, us=us)

In [ ]:
a = np.array([1,2,3])
M = np.array([[1,1,1],[1,1,1]])
a@M.T

In [ ]:
w_ = ws[0]
E@w_+w_.reshape(3,1)

In [ ]:
track_plot(
            df=df,
            lpp=ship_parameters["L"],
            beam=ship_parameters["B"],
            color="green",
        );

plot({'Simulation':df});

## Kalman filter
Implementation of the Kalman filter. The code is inspired of this Matlab implementation: [ExEKF.m](https://github.com/cybergalactic/MSS/blob/master/mssExamples/ExEKF.m).

In [ ]:
x, x1d = sp.symbols(r'\vec{x} \dot{\vec{x}}')  # State vector
h = sp.symbols('h')
u_input = sp.symbols(r'u_{input}')  # input vector
w_noise = sp.symbols(r'w_{noise}')  # input vector

f = sp.Function('f')(x,u_input,w_noise)
eq_system = sp.Eq(x1d, f)
eq_system

In [ ]:
eq_x = sp.Eq(x, sp.UnevaluatedExpr(sp.Matrix([x_0, y_0, psi, u, v, r])))
eq_x

In [ ]:
jac = sp.eye(6,6) + f_.jacobian(eq_x.rhs.doit())*h
subs = {value: key for key, value in p.items()}
subs[psi] = sp.symbols('psi')
lambda_jacobian = lambdify(jac.subs(subs))

In [ ]:
lambda_jacobian

In [ ]:
def lambda_jacobian_constructor(parameters, ship_parameters, h):
    
    def f(x, u):
        delta = u
        
        psi=x[2]
        u=x[3]
        v=x[4]
        r=x[5]
        
        jacobian = run(lambda_jacobian, **parameters, **ship_parameters, psi=psi, u=u, v=v, r=r, delta=delta, h=h)
        return jacobian
    
    return f      

In [ ]:
lambda_jacobian_ = lambda_jacobian_constructor(parameters=parameters, 
                                               ship_parameters=ship_parameters, h=h_)

In [ ]:
df_measure = df.copy()

measurement_noise_psi_max = 3
measurement_noise_psi = np.deg2rad(measurement_noise_psi_max/3)
epsilon_psi = np.random.normal(scale=measurement_noise_psi, size=N_)

measurement_noise_xy_max=2
measurement_noise_xy = measurement_noise_xy_max/3
epsilon_x0 = np.random.normal(scale=measurement_noise_xy, size=N_)
epsilon_y0 = np.random.normal(scale=measurement_noise_xy, size=N_)

df_measure['psi'] = df['psi'] + epsilon_psi
df_measure['x0'] = df['x0'] + epsilon_x0
df_measure['y0'] = df['y0'] + epsilon_y0

In [ ]:

P_prd = np.diag([0.1, 0.1, np.deg2rad(0.01), 0.001, 0.001, np.deg2rad(0.001)])
Qd = np.diag([0.01, 0.01, np.deg2rad(0.1)])  #process variances: u,v,r
Rd = h_*np.diag([measurement_noise_xy**2, measurement_noise_xy**2, measurement_noise_psi**2])  #measurement variances: x0,y0,psi

ys = df_measure[['x0','y0','psi']].values

x0_ = np.array([[0,0,0,3,0,0]]).T

Cd = np.array([
    [1, 0, 0, 0, 0, 0],
    [0, 1, 0, 0, 0, 0],
    [0, 0, 1, 0, 0, 0],
])

E = np.array([
    [0,0,0],
    [0,0,0],
    [0,0,0],
    [1,0,0],
    [0,1,0],
    [0,0,1],
    ],
)


time_steps = extended_kalman_filter(
                                    no_states=6,
                                    no_measurement_states=3,
                                    x0=x0_,
                                    P_prd=P_prd, 
                                    lambda_f=lambda_f_, 
                                    lambda_jacobian=lambda_jacobian_,
                                    h=h_, 
                                    us=us, 
                                    ys=ys, 
                                    E=E, 
                                    Qd=Qd, 
                                    Rd=Rd, 
                                    Cd=Cd)

x_hats = np.array([time_step["x_hat"].flatten() for time_step in time_steps]).T
time = np.array([time_step["time"] for time_step in time_steps]).T
Ks = np.array([time_step["K"] for time_step in time_steps]).T
variances = np.array([np.diagonal(time_step["P_hat"]) for time_step in time_steps]).T
stds = np.sqrt(variances)

In [ ]:
keys = ['x0','y0','psi','u','v','r']
fig,ax=plt.subplots()
for i,key in enumerate(keys):
    ax.plot(time, variances[i,:], label=key)
    
ax.legend()
ax.set_ylabel('std')
ax.set_xlabel('time [s]')
ax.set_ylim(0,10*np.max(variances[:,-1]))


In [ ]:
df_kalman = pd.DataFrame(data=x_hats.T, index=time, columns=['x0','y0','psi','u','v','r'])
df_kalman['delta'] = us

dataframes = {
    'Mesurement' : df_measure,
    'Kalman filter' : df_kalman,
    'Simulation' : df,
}

fig,ax=plt.subplots()
styles = {
    'Mesurement' : {
        'linestyle' : '',
        'marker' : '.',
        'ms' : 1,
    },
    
    'Kalman filter' : {
        'lw' : 2,
    },
    
    'Simulation' : {
        'lw' : 1,
        'linestyle' : ':',
    },
    
}

for label,df_ in dataframes.items():
    track_plot(
            df=df_,
            lpp=ship_parameters["L"],
            beam=ship_parameters["B"],
            ax=ax,
            label=label,
            plot_boats=False,
            **styles.get(label,{})
        );
ax.legend()


plot(dataframes = dataframes, fig_size=(10,15), styles = ['-','-',':']);

# Real data
Using the developed Kalman filter on some real model test data

## Load test

In [ ]:
id=22773
df, units, meta_data = mdl.load(dir_path = '../data/raw', id=id)
df.index = df.index.total_seconds()
df.index-=df.index[0]
df['x0']-=df.iloc[0]['x0']
df['y0']-=df.iloc[0]['y0']
df['psi']-=df.iloc[0]['psi']


In [ ]:
fig,ax=plt.subplots()
fig.set_size_inches(10,10)
track_plot(df=df, lpp=meta_data.lpp, x_dataset='x0', y_dataset='y0',  psi_dataset='psi', beam=meta_data.beam, ax=ax);

In [ ]:
sp.simplify(sp.Matrix([
    [sp.cos(psi), -sp.sin(psi)],
    [sp.sin(psi), sp.cos(psi)]]).inv())

In [ ]:
from numpy import cos as cos
from numpy import sin as sin
from src.data.lowpass_filter import lowpass_filter

df_lowpass = df.copy()
t = df_lowpass.index
ts = np.mean(np.diff(t))
fs = 1/ts

position_keys = ['x0','y0','psi']
for key in position_keys:
    df_lowpass[key] = lowpass_filter(data=df_lowpass[key], fs=fs, cutoff=1, order=1)

df_lowpass['x01d_gradient'] = x1d_ = np.gradient(df_lowpass['x0'], t)
df_lowpass['y01d_gradient'] = y1d_ = np.gradient(df_lowpass['y0'], t)
df_lowpass['r'] = r_ = np.gradient(df_lowpass['psi'], t)

psi_ = df_lowpass['psi']

df_lowpass['u'] = x1d_*cos(psi_) + y1d_*sin(psi_)
df_lowpass['v'] = -x1d_*sin(psi_) + y1d_*cos(psi_)

velocity_keys = ['u','v','r']
for key in velocity_keys:
    df_lowpass[key] = lowpass_filter(data=df_lowpass[key], fs=fs, cutoff=1, order=1)

In [ ]:
x1d_[0:10]

In [ ]:
for key in position_keys + velocity_keys:
    fig,ax=plt.subplots()
    fig.set_size_inches(12,3)
    
    df_lowpass.plot(y=key, ax=ax, zorder=-10, label='filter')
    
    if key in df:
        df.plot(y=key, ax=ax, label='raw')
        
    ax.set_ylabel(key)
    
    

In [ ]:
data = df.copy()
data['u'] = df_lowpass['u']
data['v'] = df_lowpass['v']
data['r'] = df_lowpass['r']
data=data.iloc[200:-100]
data.index-=data.index[0]

P_prd = np.diag([0.1, 0.1, np.deg2rad(0.01), 0.001, 0.001, np.deg2rad(0.001)])

Qd = np.diag([0.01, 0.01, np.deg2rad(0.1)])  #process variances: u,v,r

Cd = np.array([
    [1, 0, 0, 0, 0, 0],
    [0, 1, 0, 0, 0, 0],
    [0, 0, 1, 0, 0, 0],
])

E = np.array([
    [0,0,0],
    [0,0,0],
    [0,0,0],
    [1,0,0],
    [0,1,0],
    [0,0,1],
    ],
)


ys = data[['x0','y0','psi']].values
h_m = h_ = np.mean(np.diff(data.index))

x0_ = np.concatenate((
    data.iloc[0][['x0','y0','psi']].values,
    data.iloc[0][['u','v','r']].values))

us = data['delta'].values

error_max_pos = 0.05
sigma_pos = error_max_pos/3
variance_pos = sigma_pos**2

error_max_psi = np.deg2rad(0.5)
sigma_psi = error_max_psi/3
variance_psi = sigma_psi**2

Rd = np.diag([variance_pos, variance_pos, variance_psi])



time_steps = extended_kalman_filter(
                                    no_states=6,
                                    no_measurement_states=3,
                                    x0=x0_,
                                    P_prd=P_prd, 
                                    lambda_f=lambda_f_, 
                                    lambda_jacobian=lambda_jacobian_,
                                    h=h_, 
                                    us=us, 
                                    ys=ys, 
                                    E=E, 
                                    Qd=Qd, 
                                    Rd=Rd, 
                                    Cd=Cd)


x_hats = np.array([time_step["x_hat"].flatten() for time_step in time_steps]).T
time = np.array([time_step["time"] for time_step in time_steps]).T
Ks = np.array([time_step["K"] for time_step in time_steps]).T
variances = np.array([np.diagonal(time_step["P_hat"]) for time_step in time_steps]).T
stds = np.sqrt(variances)

In [ ]:
keys = ['x0','y0','psi','u','v','r']
fig,ax=plt.subplots()
for i,key in enumerate(keys):
    ax.plot(time, variances[i,:], label=key)
    
ax.legend()
ax.set_ylabel('std')
ax.set_xlabel('time [s]')
ax.set_ylim(0,3*np.max(variances[:,-1]))

In [ ]:
df_kalman = pd.DataFrame(data=x_hats.T, index=time, columns=['x0','y0','psi','u','v','r'])
df_kalman['delta'] = us

for key in ['u','v','r']:
    df_kalman[f'{key}1d'] = np.gradient(df_kalman[key], df_kalman.index)

dataframes = {
    'Mesurement' : data,
    'Kalman filter' : df_kalman,
}

fig,ax=plt.subplots()
styles = {
    'Mesurement' : {
        'linestyle' : '',
        'marker' : '.',
        'ms' : 1,
        'zorder':-10,
    },
    
    'Kalman filter' : {
        'lw' : 2,
    },
    
    
}

for label,df_ in dataframes.items():
    track_plot(
            df=df_,
            lpp=ship_parameters["L"],
            beam=ship_parameters["B"],
            ax=ax,
            label=label,
            plot_boats=False,
            **styles.get(label,{})
        );
ax.legend()


plot(dataframes = dataframes, 
     fig_size=(10,15), 
     styles = ['-','-',':'],
     keys=['x0','y0','psi','u','v','r','u1d','v1d','r1d']);

## RTS smoother

In [ ]:
smooth_time_steps = rts_smoother(
        time_steps=time_steps,
        us=us,
        lambda_jacobian=lambda_jacobian_,
        Qd=Qd,
        lambda_f=lambda_f_, E=E,
    )

## Post process rts smoother:
x_hats = np.array(
    [time_step["x_hat"].flatten() for time_step in smooth_time_steps]
).T
time = np.array([time_step["time"] for time_step in smooth_time_steps]).T
df_rts = pd.DataFrame(data=x_hats.T, index=time, columns=['x0','y0','psi','u','v','r'])
df_rts["delta"] = us

for key in ['u','v','r']:
    df_rts[f'{key}1d'] = np.gradient(df_rts[key], df_kalman.index)

In [ ]:
dataframes = {
    'Mesurement' : data,
    'Kalman filter' : df_kalman,
    'RTS': df_rts,
}

fig,ax=plt.subplots()
styles = {
    'Mesurement' : {
        'linestyle' : '',
        'marker' : '.',
        'ms' : 1,
        'zorder':-10,
    },
    
    'Kalman filter' : {
        'lw' : 2,
    },
    
    
}

for label,df_ in dataframes.items():
    track_plot(
            df=df_,
            lpp=ship_parameters["L"],
            beam=ship_parameters["B"],
            ax=ax,
            label=label,
            plot_boats=False,
            **styles.get(label,{})
        );
ax.legend()


plot(dataframes = dataframes, 
     fig_size=(10,15), 
     styles = ['r-','g-','b-'],
     keys=['x0','y0','psi','u','v','r','u1d','v1d','r1d']);

In [ ]:
data['thrust'] = data['Prop/PS/Thrust'] + data['Prop/SB/Thrust']
df_rts['thrust'] = data['thrust'].values
df_rts.to_csv('test.csv')

In [ ]:
smooth_time_steps[100]['P_hat']

In [ ]:
variances = np.array([np.diagonal(time_step["P_hat"]) for time_step in smooth_time_steps]).T
stds = np.sqrt(variances)

In [ ]:
keys = ['x0','y0','psi','u','v','r']
fig,ax=plt.subplots()
for i,key in enumerate(keys):
    ax.plot(time, variances[i,:], label=key)
    
ax.legend()
ax.set_ylabel('std')
ax.set_xlabel('time [s]')
ax.set_ylim(0,3*np.max(variances[:,-1]))

In [ ]:
from scipy.stats import multivariate_normal

In [ ]:
likelihoods = np.zeros(len(time_steps))
for n,smooth_time_step in enumerate(smooth_time_steps):
    cov = smooth_time_step['P_hat']
    mean = smooth_time_step['x_hat'].flatten()
    rv = multivariate_normal(mean=mean, cov=cov)
    likelihoods[n] = rv.pdf(x=mean)

In [ ]:
fig,ax=plt.subplots()

ax.plot(likelihoods)
#ax.set_ylim(2.5*10**14,3*10**14)